In [16]:
import os
# suppress silly log messages from tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
import random
import structure_prediction_utils as utils
from tensorflow import keras

In [17]:
data_folder = 'd:/UC_Master/COSC440/assignment_all/COSC440/final_project/'
training_records = utils.load_preprocessed_data(data_folder, 'training.tfr')
validate_records = utils.load_preprocessed_data(data_folder, 'validation.tfr')
test_records = utils.load_preprocessed_data(data_folder, 'testing.tfr')

Debug info, shape of one data record:
evolutionary shape= (5376,)
id shape= (1,)
primary shape= (256,)
tertiary shape= (256, 3)
primary_onehot shape= (256, 21)
mask shape= (256,)
true_distances shape= (256, 256)
distance_mask shape= (256, 256)
Value info of one data record, structure: tf.Tensor([1806.5  322.7 6882.9], shape=(3,), dtype=float32)
Debug info, shape of one data record:
evolutionary shape= (5376,)
id shape= (1,)
primary shape= (256,)
tertiary shape= (256, 3)
primary_onehot shape= (256, 21)
mask shape= (256,)
true_distances shape= (256, 256)
distance_mask shape= (256, 256)
Value info of one data record, structure: tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)
Debug info, shape of one data record:
evolutionary shape= (5376,)
id shape= (1,)
primary shape= (256,)
tertiary shape= (256, 3)
primary_onehot shape= (256, 21)
mask shape= (256,)
true_distances shape= (256, 256)
distance_mask shape= (256, 256)
Value info of one data record, structure: tf.Tensor([0. 0. 0.], shape=(3,)

In [18]:
class ProteinStructurePredictor0(keras.Model):
    def __init__(self):
        super().__init__()
        # self.layer0 = keras.layers.Conv2D(5, 5, activation='gelu', padding="same")
        self.layer1 = keras.layers.Conv2D(1, 1, activation='gelu', padding="same")

    #@tf.function
    def call(self, inputs, mask=None):
        primary_one_hot = inputs['primary_onehot']

        # outer sum to get a NUM_RESIDUES x NUM_RESIDUES x embedding size
        # x = tf.expand_dims(primary_one_hot, -2) + tf.expand_dims(primary_one_hot, -3)

        # filter the initial representation into an embedded representation
        # x = self.layer0(x)


        # add positional distance information
        r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
        distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
        distances_bc = tf.expand_dims(
            tf.broadcast_to(distances, [primary_one_hot.shape[0], utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)

        # x = tf.concat([x, x * distances_bc, distances_bc], axis=-1)
        x = distances_bc
        # generate result
        x = self.layer1(x)

        return x

In [19]:
model = ProteinStructurePredictor0()
model.optimizer = keras.optimizers.Adam(learning_rate=1e-2)
model.batch_size = 1024
epochs = 5

In [20]:
epoch_training_records = training_records.shuffle(buffer_size=256).batch(model.batch_size, drop_remainder=False)
print(epoch_training_records)

<BatchDataset shapes: {evolutionary: (None, None), id: (None, None), primary: (None, None), tertiary: (None, None, 3), primary_onehot: (None, None, 21), mask: (None, None), true_distances: (None, None, None), distance_mask: (None, None, None)}, types: {evolutionary: tf.float32, id: tf.string, primary: tf.int64, tertiary: tf.float32, primary_onehot: tf.float32, mask: tf.float32, true_distances: tf.float32, distance_mask: tf.float32}>


In [21]:
def get_n_records(batch):
    return batch['primary_onehot'].shape[0]
def get_input_output_masks(batch):
    inputs = {'primary_onehot':batch['primary_onehot']}
    outputs = batch['true_distances']
    masks = batch['distance_mask']
    return inputs, outputs, masks

In [22]:
print(epoch_training_records)

<BatchDataset shapes: {evolutionary: (None, None), id: (None, None), primary: (None, None), tertiary: (None, None, 3), primary_onehot: (None, None, 21), mask: (None, None), true_distances: (None, None, None), distance_mask: (None, None, None)}, types: {evolutionary: tf.float32, id: tf.string, primary: tf.int64, tertiary: tf.float32, primary_onehot: tf.float32, mask: tf.float32, true_distances: tf.float32, distance_mask: tf.float32}>


In [23]:
epoch_training_records = training_records.shuffle(buffer_size=256).batch(model.batch_size, drop_remainder=False)
i = 0
for batch in epoch_training_records:
    inputs, labels, masks = get_input_output_masks(batch)
    i += 1
    print("----------")
    print(inputs['primary_onehot'].shape)
    print(labels.shape)
    print(masks.shape)


----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(1024, 256, 21)
(1024, 256, 256)
(1024, 256, 256)
----------
(736, 256, 21)
(736, 256, 256)
(736, 256, 256)


In [24]:
print(i)
print(inputs['primary_onehot'].shape)
print(labels.shape)
print(masks.shape)

13
(736, 256, 21)
(736, 256, 256)
(736, 256, 256)


In [25]:
r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
print(distances.shape)
distances_bc = tf.expand_dims(
    tf.broadcast_to(distances, [256, utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)
print(distances_bc.shape)

(256, 256, 256, 1)
